In [9]:
import numpy as np
import csv

#*** TRAINING ***
train_file = open('covid_training.tsv', 'r', encoding='utf-8')
tweets = {}
vocabulary = []

next(train_file) # skip header

for line in train_file:
    # split tsv lines
    split_line = line.split("\t")
    
    # get tweet data and fold to lower case
    tweet_id = split_line[0]
    tweet_text = split_line[1].lower()
    q_label = split_line[2]
    
    # get tweet text
    split_text = tweet_text.split(" ")
    tweet_vector = {}
    for token in split_text:
        # add term to tweet_vector and count frequency
        if token not in tweet_vector.keys():
            vocabulary.append(token)
            tweet_vector[token] = 1
        else:
            tweet_vector[token] += 1    
            
    # add tweet vector and label to tweets dictionary
    tweets[tweet_id] = (tweet_vector, q_label)
    
# no duplicates
vocabulary = sorted(set(vocabulary))
#print(vocabulary)


In [68]:
# *** Ignore this cell (it's for testing) ***
# tweets directory{}, and vocabulary list[]
#for i in tweets:
#    if tweets[i][1] == "no":
#        words = list(tweets[i][0])
#        zipped = zip(tweets[i][0].keys(), tweets[i][0].values())
#        print( list(zipped) )
frequencies_for_c1 = []
frequencies_for_c2 = []
words_only_c1 = []
for i in tweets:  
    if tweets[i][1] == "yes":
            words = list(tweets[i][0].keys())
            values = list(tweets[i][0].values())
            pairs = zip(words,values)
            for entry in pairs:
                frequencies_for_c1.append(entry)
    elif tweets[i][1] == "no":
            words = list(tweets[i][0].keys())
            values = list(tweets[i][0].values())
            pairs = zip(words,values)
            for entry in pairs:
                frequencies_for_c2.append(entry)   
print(frequencies_for_c1)
print(len(frequencies_for_c1))
print('\n')
print(frequencies_for_c2)
print(len(frequencies_for_c2))

        
test_example = 'Panic buying and stockpiling of toilet roll continues. These are the scenes at Costco in Farnborough in Hampshire today. #coronavirus #toiletpaperpanic #panickbuying https://t.co/JLlZVVS7EH'.lower()
splited_text = test_example.split(" ")
print('\n', splited_text)        

sum = 0
for entry in frequencies_for_c1:
    if splited_text[0] in vocabulary:
        if (entry[0] == splited_text[0]) or (entry[0] == splited_text[0]+',') or (entry[0] == splited_text[0]+'.'):
            sum+=entry[1]
            
print(sum)
#print(len(tweets))
    
#print(len(vocabulary))
#print(vocabulary)


[('this', 1), ('is', 1), ('the', 2), ('face', 1), ('of', 1), ('someone', 1), ('who', 1), ('just', 1), ('spent', 1), ('9', 1), ('hours', 1), ('in', 1), ('personal', 1), ('protective', 1), ('equipment', 1), ('moving', 1), ('critically', 1), ('ill', 1), ('covid19', 1), ('patients', 1), ('around', 1), ('london.', 1), ('', 1), ('i', 2), ('feel', 1), ('broken', 1), ('-', 1), ('and', 2), ('we', 1), ('are', 1), ('only', 1), ('at', 1), ('start.', 1), ('am', 1), ('begging', 1), ('people,', 1), ('please', 2), ('do', 1), ('social', 1), ('distancing', 1), ('self', 1), ('isolation', 1), ('#covid19', 1), ('https://t.co/hs0rqdvsn3', 1), ('yesterday', 1), ('i', 2), ('knocked', 1), ('on', 1), ('my', 1), ("tennant's", 1), ('door', 1), ('and', 1), ('told', 1), ('him', 1), ('he', 2), ('can', 1), ('stay', 1), ('in', 1), ('the', 1), ('property', 1), ('rent', 1), ('free', 1), ('for', 1), ('5', 1), ('months.', 1), ('is', 2), ('self', 1), ('employed', 1), ('with', 1), ('4', 1), ('kids.', 1), ("don't", 1), ('nee

8


In [104]:
# *** Compute Probabilities and TESTING ***

#0. Count the unique set of words present in each class
count_set_c1 = 0 # count set of words present when class is 'yes' - will be used in the conditional probabilities
words_frequency_c1 = [] # words in class c1 with frequencies (from all tweets)
words_c1 = [] # only words with no frequencies (from all tweets)
data_instances_c1 = 0 # number of data instances with class 'yes' (from all tweets)

count_set_c2 = 0 # count set of words present when class is 'no' - will be used in the conditional probabilities
words_frequency_c2 = []  # words in class c2 with frequencies (from all tweets)
words_c2 = [] # only words with no frequencies (from all tweets)
data_instances_c2 = 0 # number of data instances with class 'no' (from all tweets)

for i in tweets:
    # c1: Check each tweet where q1 label=yes
    if tweets[i][1] == "yes":
        data_instances_c1+=1
        words = list(tweets[i][0].keys())
        values = list(tweets[i][0].values())
        pairs = zip(words,values)
        for entry in pairs:
            words_frequency_c1.append(entry)
            words_c1.append(entry[0])
            
    # c2: Check each tweet where q1 label=no        
    elif tweets[i][1] == "no":
        data_instances_c2+=1
        words = list(tweets[i][0].keys())
        values = list(tweets[i][0].values())
        pairs = zip(words,values)
        for entry in pairs:
            words_frequency_c2.append(entry)   
            words_c2.append(entry[0])
        
count_set_c1 = len(set(words_c1)) # c1 set - no duplicates even from other tweets
#print(words_frequency_c1)
#print(len(words_c1))
#print(count_set_c1)
#print(data_instances_c1)
count_set_c2 = len(set(words_c2)) # c2 set - no duplicates even from other tweets
#print(words_frequency_c2)
#print(len(words_c2))
#print(count_set_c2)
#print(data_instances_c2)

#1. Prior Probabilities
# How many instances of that class, over all the tweets
number_tweets = len(tweets)
priorprob_c1 = data_instances_c1/number_tweets
priorprob_c2 = data_instances_c2/number_tweets

#2. TEST data, Conditional Probabilities and SCORE(class)
# notice that some words in the tweet may not be part of the vocabulary
# TEST DATA
test_file = open('covid_test_public.tsv', 'r', encoding='utf-8')
test_tweets = {}
for line in test_file:
    # split tsv lines
    split_line = line.split("\t")
    # get tweet data and fold to lower case
    tweet_id = split_line[0]
    tweet_text = split_line[1].lower()
    expected_q_label = split_line[2]
    test_tweets[tweet_id] = (tweet_text, expected_q_label)

print(len(test_tweets))
# TESTING TWEETS
for key in test_tweets.keys():
    test_example = test_tweets[key][0]
    expected_class = test_tweets[key][1]
    
    #test_example = 'Panic buying and stockpiling of toilet roll continues. These are the scenes at Costco in Farnborough in Hampshire today. #coronavirus #toiletpaperpanic #panickbuying https://t.co/JLlZVVS7EH'.lower()
    print(test_example)
    splited_text = test_example.split(" ")


    # We don't compute all the conditional probabilities... only do so when determining score() for new tweet
    #TO-DO: ADD smoothing and log10
    delta = 0.01
    # *** SCORE(c1) - class 'yes' ***
    conditionalprob_c1 = []
    for word in splited_text:
        word_count = 0 # count frequency for each word in the class c1
        if word in vocabulary: #find the conditional probability of this word and add it to the list
            for entry in words_frequency_c1: # find the frequency of the word in all tweets with class c1
                if (entry[0] == word) or (entry[0] == word+'.') or (entry[0] == word+',') or (entry[0] == word+';'):
                    word_count+=entry[1]
            #add the conditional probability
            conditionalprob_c1.append(word_count/count_set_c1)
            print(word_count)
            print((word_count/count_set_c1), '\n')
    #SCORE(c1)
    score_c1 = 1
    for probability in conditionalprob_c1:
         score_c1 = score_c1 * probability 
    # Finally, multiply by the Prior probability
    score_c1 = score_c1 * priorprob_c1;
    print("score(c1): ", score_c1)

    # *** SCORE(c2) - class 'no' ***
    conditionalprob_c2 = []
    for word in splited_text:
        word_count = 0 # count frequency for each word in the class c2
        if word in vocabulary: #find the conditional probability of this word and add it to the list
            for entry in words_frequency_c2: # find the frequency of the word in all tweets with class c2
                if (entry[0] == word) or (entry[0] == word+'.') or (entry[0] == word+',') or (entry[0] == word+';'):
                    word_count+=entry[1]
            #add the conditional probability
            conditionalprob_c2.append(word_count/count_set_c2)
            print(word_count)
            print((word_count/count_set_c2), '\n')
    #SCORE(c2)
    score_c2 = 1
    for probability in conditionalprob_c2:
         score_c2 = score_c2 * probability 
    # Finally, multiply by the Prior probability
    score_c2 = score_c2 * priorprob_c2;
    print("score(c2): ", score_c2)

    # argmax(score_c1, score_c2)
    if max(score_c1, score_c2) == score_c1:
        print("The class of that tweet is c1: yes ")
        prediction = "yes"
        score = score_c1
    else:
        print("The class of that tweet is c2: no")
        prediction = "no"
        score = score_c2
        
    # WHAT WAS THE EXPECTED CLASS?
    print("Expected class: ", expected_class)
    
    # Correct or Wrong prediction?
    if prediction == expected_class:
        result = "correct"
    else:
        result = "wrong"
    
    #4. WRITE TRACE FILES
    outFile = open('trace_NB-BOW-OV.txt', "a")
    outFile.write(key)
    outFile.write(", ")
    outFile.write(prediction)
    outFile.write(", ")
    outFile.write(str(score))
    outFile.write(", ")
    outFile.write(expected_class)
    outFile.write(", ")
    outFile.write(result)
    outFile.write("\n")
    
outFile.close()

55
1/ many of you ask me why i take the covid-19 outbreak so seriously. current numbers of cases and deaths are *not* why. ð a thread on why iâm worried and what i do personally in this situation.
0
0.0 

11
0.0033102618116160097 

180
0.05416792055371652 

55
0.016551309058080048 

6
0.0018055973517905506 

7
0.0021065302437556425 

7
0.0021065302437556425 

49
0.014745711706289497 

12
0.0036111947035811012 

320
0.09629852542882937 

66
0.019861570869696057 

9
0.002708396027685826 

16
0.004814926271441469 

1
0.0003009328919650918 

2
0.0006018657839301836 

4
0.0012037315678603672 

180
0.05416792055371652 

32
0.009629852542882938 

159
0.047848329822449596 

6
0.0018055973517905506 

52
0.015648510382184774 

181
0.054468853445681614 

2
0.0006018657839301836 

69
0.020764369545591334 

7
0.0021065302437556425 

1
0.0003009328919650918 

159
0.047848329822449596 

15
0.0045139933794763765 

49
0.014745711706289497 

14
0.004213060487511285 

137
0.04122780619921757 

79
0.

36
0.010833584110743304 

4
0.0012037315678603672 

244
0.0734276256394824 

0
0.0 

3
0.0009027986758952753 

2
0.0006018657839301836 

score(c1):  0.0
6
0.0036496350364963502 

22
0.01338199513381995 

8
0.004866180048661801 

1
0.0006082725060827251 

2
0.0012165450121654502 

0
0.0 

1
0.0006082725060827251 

50
0.030413625304136254 

0
0.0 

0
0.0 

8
0.004866180048661801 

16
0.009732360097323601 

0
0.0 

93
0.05656934306569343 

1
0.0006082725060827251 

0
0.0 

0
0.0 

score(c2):  0.0
The class of that tweet is c1: yes 
Expected class:  yes
why is cdc's data on us cases lagging behind other entities (including who)? why are we only learning from the ap about a reported cdc recommendation that older americans not fly, and that the white house blocked cdc from publicizing that recommendation? https://t.co/6j1cael8zr
7
0.0021065302437556425 

140
0.04213060487511285 

1
0.0003009328919650918 

69
0.020764369545591334 

21
0.006319590731266927 

32
0.009629852542882938 

1
0.00030

50
0.030413625304136254 

0
0.0 

10
0.006082725060827251 

1
0.0006082725060827251 

3
0.0018248175182481751 

50
0.030413625304136254 

0
0.0 

0
0.0 

23
0.013990267639902677 

0
0.0 

0
0.0 

0
0.0 

2
0.0012165450121654502 

1
0.0006082725060827251 

score(c2):  0.0
The class of that tweet is c1: yes 
Expected class:  yes
toilet paper completely sold out in supermarkets? did you lot not wipe your ass before corona
2
0.0006018657839301836 

2
0.0006018657839301836 

1
0.0003009328919650918 

10
0.003009328919650918 

137
0.04122780619921757 

10
0.003009328919650918 

55
0.016551309058080048 

1
0.0003009328919650918 

52
0.015648510382184774 

19
0.005717724947336744 

0
0.0 

3
0.0009027986758952753 

42
0.012639181462533854 

score(c1):  0.0
0
0.0 

1
0.0006082725060827251 

0
0.0 

13
0.007907542579075427 

44
0.0267639902676399 

2
0.0012165450121654502 

51
0.03102189781021898 

3
0.0018248175182481751 

20
0.012165450121654502 

15
0.009124087591240875 

5
0.0030413625304136

2
0.0006018657839301836 

137
0.04122780619921757 

2
0.0006018657839301836 

1
0.0003009328919650918 

1
0.0003009328919650918 

1
0.0003009328919650918 

score(c1):  5.5745789676272404e-67
0
0.0 

1
0.0006082725060827251 

0
0.0 

93
0.05656934306569343 

4
0.0024330900243309003 

10
0.006082725060827251 

36
0.021897810218978103 

4
0.0024330900243309003 

5
0.0030413625304136255 

0
0.0 

14
0.00851581508515815 

39
0.023722627737226276 

9
0.005474452554744526 

132
0.08029197080291971 

0
0.0 

14
0.00851581508515815 

12
0.0072992700729927005 

26
0.015815085158150853 

0
0.0 

1
0.0006082725060827251 

0
0.0 

44
0.0267639902676399 

0
0.0 

0
0.0 

0
0.0 

0
0.0 

score(c2):  0.0
The class of that tweet is c1: yes 
Expected class:  yes
corona has no chance against my essential oils mom.
42
0.012639181462533854 

50
0.01504664459825459 

24
0.0072223894071622025 

9
0.002708396027685826 

22
0.0066205236232320195 

1
0.0003009328919650918 

1
0.0003009328919650918 

score(c1): 

244
0.0734276256394824 

159
0.047848329822449596 

90
0.02708396027685826 

79
0.02377369846524225 

2
0.0006018657839301836 

45
0.01354198013842913 

3
0.0009027986758952753 

score(c1):  0.0
20
0.012165450121654502 

2
0.0012165450121654502 

26
0.015815085158150853 

17
0.010340632603406326 

132
0.08029197080291971 

20
0.012165450121654502 

8
0.004866180048661801 

93
0.05656934306569343 

4
0.0024330900243309003 

0
0.0 

46
0.027980535279805353 

63
0.03832116788321168 

11
0.006690997566909975 

0
0.0 

0
0.0 

1
0.0006082725060827251 

0
0.0 

1
0.0006082725060827251 

20
0.012165450121654502 

3
0.0018248175182481751 

68
0.0413625304136253 

2
0.0012165450121654502 

93
0.05656934306569343 

68
0.0413625304136253 

34
0.02068126520681265 

46
0.027980535279805353 

0
0.0 

29
0.017639902676399026 

0
0.0 

score(c2):  0.0
The class of that tweet is c1: yes 
Expected class:  yes
how should we approach potential covid-19 cases? here's a infographic i made based on the 09 ma

39
0.011736382786638579 

55
0.016551309058080048 

244
0.0734276256394824 

19
0.005717724947336744 

4
0.0012037315678603672 

2
0.0006018657839301836 

31
0.009328919650917845 

3
0.0009027986758952753 

72
0.021667168221486607 

50
0.01504664459825459 

1
0.0003009328919650918 

6
0.0018055973517905506 

180
0.05416792055371652 

1
0.0003009328919650918 

38
0.011435449894673488 

77
0.02317183268131207 

180
0.05416792055371652 

37
0.011134517002708396 

26
0.007824255191092387 

34
0.01023171832681312 

5
0.001504664459825459 

3
0.0009027986758952753 

13
0.003912127595546194 

38
0.011435449894673488 

3
0.0009027986758952753 

14
0.004213060487511285 

320
0.09629852542882937 

11
0.0033102618116160097 

6
0.0018055973517905506 

3
0.0009027986758952753 

1
0.0003009328919650918 

90
0.02708396027685826 

1
0.0003009328919650918 

score(c1):  7.062954395695918e-81
5
0.0030413625304136255 

26
0.015815085158150853 

51
0.03102189781021898 

93
0.05656934306569343 

15
0.009124

me explaining to my kids how i survived corona virus https://t.co/wviiwqwhvm
7
0.0021065302437556425 

244
0.0734276256394824 

22
0.0066205236232320195 

2
0.0006018657839301836 

6
0.0018055973517905506 

49
0.014745711706289497 

1
0.0003009328919650918 

42
0.012639181462533854 

48
0.014444778814324405 

score(c1):  5.581223151851108e-22
8
0.004866180048661801 

93
0.05656934306569343 

18
0.010948905109489052 

2
0.0012165450121654502 

15
0.009124087591240875 

37
0.022506082725060828 

1
0.0006082725060827251 

39
0.023722627737226276 

28
0.0170316301703163 

score(c2):  7.049290874816104e-20
The class of that tweet is c2: no
Expected class:  no
the country is panic stricken over the #coronavirus, yet this flu season there have been 9.7 million flu illnesses, 87,000 hospitalizations and 4800 deaths from flu, including 32 pediatric deaths. itâs time to stop the panic and put things into perspective.
320
0.09629852542882937 

5
0.001504664459825459 

140
0.04213060487511285 



137
0.04122780619921757 

7
0.0021065302437556425 

5
0.001504664459825459 

45
0.01354198013842913 

3
0.0009027986758952753 

181
0.054468853445681614 

180
0.05416792055371652 

37
0.011134517002708396 

13
0.003912127595546194 

1
0.0003009328919650918 

27
0.008125188083057478 

7
0.0021065302437556425 

13
0.003912127595546194 

320
0.09629852542882937 

2
0.0006018657839301836 

180
0.05416792055371652 

320
0.09629852542882937 

320
0.09629852542882937 

2
0.0006018657839301836 

1
0.0003009328919650918 

1
0.0003009328919650918 

score(c1):  3.26157950251342e-88
37
0.022506082725060828 

0
0.0 

46
0.027980535279805353 

0
0.0 

7
0.004257907542579075 

37
0.022506082725060828 

7
0.004257907542579075 

18
0.010948905109489052 

93
0.05656934306569343 

8
0.004866180048661801 

15
0.009124087591240875 

23
0.013990267639902677 

63
0.03832116788321168 

93
0.05656934306569343 

2
0.0012165450121654502 

8
0.004866180048661801 

0
0.0 

34
0.02068126520681265 

10
0.00608272506